<a href="https://colab.research.google.com/github/Deminalla/Kursinis/blob/main/fta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install scikit-learn

!pip install bitsandbytes
!pip install accelerate

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ekperimentas 1
# llama3 modelis geras lietuviu kalbai
# suskaiciuoti tikslumas, darant klasifikacija
# use prompts/instrukcija
https://www.kaggle.com/code/lucamassaron/fine-tune-llama-3-for-sentiment-analysis
# modelio pacio nekeiciam ir nemokinam, tsg jam duodam duomenis ir jis klasifikuoja kaip moka

# ekperimentas 2
# transfer learning arba fine-tuning
https://huggingface.co/docs/transformers/tasks/sequence_classification
# cia vyks mokymas

# galesim palygint situos 2 atvejus ir palygint ju statistikas, pvz tikslumas, F1 ir t.t.

# ekperimentas 3
# gebetu identifikuot kur yra konkretus fta pastraipoje
# rodyti indeksus kur prasidejo ir baigesi, jeigu nfta tai 0, 0
# reiks keist exceli, kad rodyt pradzios intervala ir pabaigos

# ekperimentas 4
# naudot lietuviska llama2 kuris jau pamokytas lietuviu kalbai
# ...

In [6]:
import requests

url = 'https://github.com/Deminalla/Kursinis/blob/main/FTA_dataset.xlsx'
response = requests.get(url)

with open('FTA_dataset.xlsx', 'wb') as f:
    f.write(response.content)

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

file_path = 'FTA_dataset.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='FTA', header=None)
sheet2 = pd.read_excel(file_path, sheet_name='NFTA', header=None)

sheet1.columns = ['paragraph']
sheet2.columns = ['paragraph']
sheet1['fta'] = 1
sheet2['fta'] = 0

data = pd.concat([sheet1, sheet2], ignore_index=True)

# Shuffle the data
# sample() - randomly select data
# frac=1 - select 100% of data
# After shuffling, the original index of the DataFrame is no longer in order
# reset_index(drop=True) is used to reset the index to a new sequential order.
# drop=True - discards the old index
data = data.sample(frac=1).reset_index(drop=True)

# training (70%), validation (10%), and test (20%)
train, temp = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=(2/3), random_state=42)

In [ ]:
!pip install XlsxWriter

output_file_path = 'FTA_split_dataset.xlsx'

# save split data so it doesnt get lost or modified during experiments
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    train.to_excel(writer, sheet_name='Train', index=False)
    val.to_excel(writer, sheet_name='Validation', index=False)
    test.to_excel(writer, sheet_name='Test', index=False)

Eksperimentas 1

In [ ]:
y_true = test["fta"].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("Undi95/Meta-Llama-3-8B-hf")
model = AutoModelForCausalLM.from_pretrained("Undi95/Meta-Llama-3-8B-hf",
                                             load_in_8bit=True, # quantization_config=BitsAndBytesConfig(load_in_8bit=True)
                                             device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
def generate_prompt(data_point):
    return f"""
            Sociolingvistikos srityje Įvaizdžiui Grėsmingas Šnekos Aktas (ĮGŠA) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui.
            Pavyzdžiai ĮGŠA yra įžeidimai, pertraukimai ir kritika.
            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Įvaizdžiui Grėsmingas Šnekos Aktas.
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [{data_point["paragraph"]}] = """

In [ ]:
X_test = pd.DataFrame(test.apply(generate_prompt, axis=1), columns=["paragraph"])

In [ ]:
from tqdm import tqdm

def predict(model, tokenizer):
    y_pred = []
    pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 1,
                do_sample=False
                )
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["paragraph"]
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        print("paragraph: ", test.iloc[i]["paragraph"])
        print("answer: ", answer)
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
             y_pred.append(0)
        else:
            y_pred.append(-1)
    return y_pred

In [ ]:
y_pred = predict(model, tokenizer)

  0%|          | 0/143 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|          | 1/143 [00:00<01:25,  1.67it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


paragraph:  Taip. Tai aš dabar ir nebesuprantu, ką mes turim šnekėt, ar apie kovą su korupcija, ar apie migraciją vidinę.
answer:   1


  1%|▏         | 2/143 [00:01<01:13,  1.91it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


paragraph:  Aš noriu, kad į mūsų diskusiją įsiterptų profesorius Antanas Kulakauskas, kuris tom temom yra skyręs daug akademinio dėmesio.
answer:   1


  2%|▏         | 3/143 [00:01<01:10,  1.98it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

paragraph:  Sveiki. Naujamiesčio apygardos kandidatas, eem, tenka, ypatingai pastaruoju metu, daug keliauti po svečias šalis ir susitikti su lietuviais, tai, iš tikrųjų, jie yra nusivylę tuo, kad dviguba pilietybė, svarstymas ir būtent net internetinio balsavimo klausimas yra ilgai svarstomas, bet niekas negali eem, priimti jo, tai liberalai vieninteliai, kurie nuosekliai politiką būtent tuo klausimu vykdėo. Tai, iš tikrųjų, tai padėtų su jais palaikyti geresnį ryšį ir kad jie galėtų dalyvauti, būtent, politiniam gyvenime Lietuvos.
answer:   1


 99%|█████████▉| 142/143 [01:06<00:00,  2.33it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


paragraph:  Galima pasakyt, kad bendrai, nuo 2010-ųjų metų šilumos ūkis iš esmės kontroliuojamas ir valdomas valstybės ir šiuo atveju tokios valsybinės įmonės, gana šiandien tapusios labai didele, tai yra “Lietuvos energija”. Esminiai sprendimai priimami būtent Vyriausybėje, Energetikos ministeriijoje ir savivaldybių įtaka čia praktiškai yra nulinė.
answer:   1


100%|██████████| 143/143 [01:06<00:00,  2.14it/s]

paragraph:  Dėl kolegos pasiūlymo dėl darbo vietų. Jeigu Darbo kodeksas bus toks, koks šiandien, tai jokių darbo vietų nebus, žmonės emigruos ir dar smarkiau. Apie tai smulkiau galėtų mūsų koalicijos partneris ir narys, Profsajungų centro pirmininkas Kęstutis Juknis.
answer:   1


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred):
    # Filter out undetermined predictions
    mask = [y != -1 for y in y_pred]
    filtered_y_true = [y_true[i] for i in range(len(y_true)) if mask[i]]
    filtered_y_pred = [y_pred[i] for i in range(len(y_pred)) if mask[i]]

    accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
    precision = precision_score(filtered_y_true, filtered_y_pred, pos_label=1)
    recall = recall_score(filtered_y_true, filtered_y_pred, pos_label=1)
    f1 = f1_score(filtered_y_true, filtered_y_pred, pos_label=1)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

In [ ]:
evaluate(y_true, y_pred)

Accuracy: 0.26573426573426573
Precision: 0.23529411764705882
Recall: 0.9696969696969697
F1 Score: 0.378698224852071


Eksperimentas 2